In [32]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser


from dotenv import load_dotenv
from pyprojroot import here
from langsmith import Client
from uuid import uuid4

import os
## INICIAR LANGSMITH Y API KEYS
dotenv_path = here() / ".env"
load_dotenv(dotenv_path=dotenv_path)


client = Client()

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_PROJECT"] = f"Debug - {unique_id}"





prompt_gen_resumen = PromptTemplate(template=""" 
Genera un resumen enfocado y detallado, limitado a aproximadamente 100 palabras, sobre la seccion {seccion} del {manual} de Bantotal, siguiendo estas directrices:

- Iniciar directamente con una descripción específica de los procesos, técnicas, o aspectos relevantes cubiertos en la sección, omitiendo cualquier introducción o mención general al manual.
- Incluir un breve resumen de las subsecciones, si están presentes, destacando sus contribuciones clave al tema principal.
- Proporcionar una descripción concisa de cualquier tabla incluida, enfocándose en su función y relevancia dentro del contexto de migración. Estas se incluyen como TABLA: [tabla]
El contenido a resumir está delimitado por triple comillas (''').
                                    
Contenido: ''' 
{contenido}                         
'''
Resumen de 100 palabras:
""",input_variables=["contenido","seccion","manual"])


In [10]:
import docx
import re
from langchain.docstore.document import Document
import os


import docx
import os



class tree():
    def __init__(self, section,level,manual):
        self.section = section
        self.childrens = []
        self.level = level
        self.resumen = ""
        self.content = ""
        self.manual = manual

    def add_child(self,child):  
        self.childrens.append(child)

    def print_index(self, numbering="", indent=0, index_string="",w_content=False):
        index_string += '\t' * indent + numbering + self.section+"\n"
        for i, child in enumerate(self.childrens, start=1):
            index_string = child.print_index(numbering + f"{i}.", indent + 1, index_string)
        return index_string

    def print_index_w_content(self, numbering="", indent=0, index_string=""):
        index_string += '\t' * indent + numbering + self.section + ':  '+ self.resumen +"\n"
        for i, child in enumerate(self.childrens, start=1):
            index_string = child.print_index_w_content(numbering + f"{i}.", indent + 1, index_string)
        return index_string

    def generar_resumenes_input(self):
        for i, child in enumerate(self.childrens, start=1):
            child_resumen = child.generar_resumenes_input()
        self.resumen = input(f"Manual: {self.manual}, Seccion: {self.section}")
        return self.resumen
    
    def generar_resumenes_AI(self,numbering=""):
        full_content = numbering + self.section + ": "+self.content + '\n'
        if len(self.childrens) != 0:
            full_content += '\n \n Resumen subsecciones: \n'
        for i, child in enumerate(self.childrens, start=1):
            child_resumen = child.generar_resumenes(numbering=numbering + f"{i}.")
            full_content += '\n' + numbering + f"{i} " + child.section + ": "+child.resumen + '\n'
        chain = prompt_gen_resumen | ChatOpenAI(temperature=0.0) | StrOutputParser()
        self.resumen = chain.invoke({"contenido":full_content,"seccion": numbering + self.section,"manual":self.manual})
        return self.resumen

    def lista_secc_y_desc(self, numbering=""):
        lista_sec = []
        lista_desc = []
        for i, child in enumerate(self.childrens, start=1):
            lista_sec_i, lista_desc_i = child.lista_secc_y_desc(numbering=numbering + f"{i}.")
            lista_sec += lista_sec_i
            lista_desc += lista_desc_i
        lista_sec.append(numbering + " " + self.section )
        lista_desc.append(self.resumen)
        return lista_sec, lista_desc

    def get_resumenes_y_content_list(self):
        lista_resumenes = []
        lista_content = []
        for i, child in enumerate(self.childrens, start=1):
            lista_resumenes_i, lista_content_i = child.get_resumenes_y_content_list()
            lista_resumenes += lista_resumenes_i
            lista_content += lista_content_i
        lista_resumenes.append(self.resumen)
        lista_content.append(self.content)
        return lista_resumenes, lista_content

class docx_loader():
    def __init__(self, docx_path):
        self.documents = []
        self.docx_path = docx_path
        self.manual = os.path.splitext(os.path.basename(docx_path))[0]
        self.contenido = tree(self.manual,0,self.manual) 

    def split_by_sections(self):
        docx_path = self.docx_path
        doc = docx.Document(docx_path)
        documents = []
        current_metadata = {}
        current_content = []
        level = 0
        # Obtener el título del documento a partir del nombre del archivo
        current_metadata['titulo'] = os.path.splitext(os.path.basename(docx_path))[0]

        for block in self.iter_block_items(doc):
            if isinstance(block, docx.text.paragraph.Paragraph):
                paragraph = block
                if paragraph.style.name.startswith('Heading') or paragraph.style.name.startswith('Titulo'):
                    # Guardar el documento actual si hay contenido
                    page_content = '\n'.join(current_content)
                    documents.append(Document(page_content=page_content, metadata=current_metadata.copy()))
                    self.add_contenido_a_nodo(current_metadata, level, page_content)
                    current_content = []

                    # Extraer nivel y texto del encabezado
                    level = int(float(paragraph.style.name.split(' ')[1]))
                    text = re.sub(r'^[\d.]+\s*', '', paragraph.text)
                    current_metadata[f's{level}'] = text
                    for i in range(level + 1, len(current_metadata) + 1):
                        current_metadata.pop(f's{i}', None)

                    # Añadir el título al árbol de contenido
                    if not text.strip() is "":
                        self.add_to_contenido(current_metadata, level)

                else:
                    if paragraph.text.strip() != '':
                        current_content.append(paragraph.text)

            elif isinstance(block, docx.table.Table):
                table = block
                texto_tabla = ""
                texto_tabla += 'TABLA: \n'
                for row in table.rows:
                    for cell in row.cells:
                        texto_tabla += self.clean_document(cell.text) + ' | '
                    texto_tabla += '\n'
                current_content.append(texto_tabla)

        # Asegurarse de añadir el último documento
        if current_content:
            page_content = '\n'.join(current_content)
            documents.append(Document(page_content=page_content, metadata=current_metadata.copy()))
        if os.path.splitext(os.path.basename(docx_path))[0] != "Manual Proceso de Migracion":
            content = self.contenido.print_index().replace("Manual", "Manual Migracion de")
            documents.insert(1,Document(page_content=content, metadata={"titulo":os.path.splitext(os.path.basename(docx_path))[0], "s1":"Contenido"}))
        else:
            documents.insert(1,Document(page_content=self.contenido.print_index(), metadata={"titulo":os.path.splitext(os.path.basename(docx_path))[0], "s1":"Contenido"}))
        return documents

    def add_to_contenido(self,  current_metadata, level):
        """
        Agrega un elemento al árbol de contenido.
        """
        current_node = self.contenido
        for i in range(1, level):
            for child in current_node.childrens:
                if current_metadata[f's{i}'] == child.section:
                    current_node = child
        new_node = tree(current_metadata[f's{level}'],level,self.manual)
        current_node.childrens.append(new_node)

    def add_contenido_a_nodo(self,  current_metadata, level,content):
        """
        Agrega un elemento al árbol de contenido.
        """
        current_node = self.contenido
        for i in range(1, level+1):
            for child in current_node.childrens:
                if current_metadata[f's{i}'] == child.section:
                    current_node = child
        current_node.content = content

    def iter_block_items(self, doc):
        """
        Genera un iterador de todos los elementos del cuerpo del documento, 
        incluyendo párrafos y tablas.
        """
        for item in doc.element.body:
            if isinstance(item, docx.oxml.text.paragraph.CT_P):
                yield docx.text.paragraph.Paragraph(item, doc._part)
            elif isinstance(item, docx.oxml.table.CT_Tbl):
                yield docx.table.Table(item, doc._part)

    def clean_document(self,document):
        cleaned_document = re.sub(r'\n+', '\n', document.replace("\r"," ").replace("\t"," ")).strip()
        cleaned_document = cleaned_document.replace("=", " ").replace("-", " ")
        cleaned_document = re.sub(r'\s+', ' ', cleaned_document).strip()
        cleaned_document = (" ").join(cleaned_document.split(" ")) #[:512]
        return cleaned_document

<>:110: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:110: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\iboero\AppData\Local\Temp\ipykernel_17524\2799280292.py:110: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if not text.strip() is "":


In [11]:
in_path = 'C:/Users/iboero/Chat_w_PDF/data/raw_docs/Migracion/'

In [ ]:
# System imports
import os
import json
import dill as pkl


# Cargar documentos

trees = {}
docs = {}
resumenes = {}
for raiz, dirs, archivos in os.walk(in_path):
        for archivo in archivos:
            if archivo.endswith(".docx"):
                path_completo = os.path.join(raiz, archivo)
                loader = docx_loader(path_completo)
                manual = archivo[:-5]
                secciones = loader.split_by_sections()
                loader.contenido.generar_resumenes_input()
                resumenes[manual], docs[manual] = loader.contenido.get_resumenes_y_content_list()
                trees[manual] = loader.contenido



with open("resumenes.txt", "w", encoding="utf-8") as archivo:
    json.dump(resumenes, archivo, ensure_ascii=False)

with open("documentos.txt", "w", encoding="utf-8") as archivo:
    json.dump(docs, archivo, ensure_ascii=False)

filename = 'dict_arbol_secciones.pkl'
with open(filename, 'wb') as file:
    pkl.dump(trees, file)


In [23]:
def strip_seccion(tree):
    tree.section = tree.section.strip()
    for i, child in enumerate(tree.childrens, start=1):
        strip_seccion(child)
    return 


In [22]:
for k in trees.keys():
    strip_seccion(trees[k])
    print(trees[k].print_index_w_content())

Manual Acuerdos de Sobregiro:  
	1.Ciclo de Migraciones:  No tiene contenido, sirve para agrupar subsecciones hijas
		1.1.Introducción:  Explica brevemente el propósito de la migración de y los pasos generales involucrados, incluyendo la generación de estructuras en bandejas y el control de calidad de datos.
		1.2.Migraciones Previas:  Discute los requisitos de migraciones previas, como aquellos sistemas necesarios de migrar previamente.
			1.2.1.Migración de Saldos Iniciales:  
			1.2.2.Migración de Clientes:  
			1.2.3.Migración de Cuentas Vista:  
		1.3.Generación de Estructuras de Bandejas:  Lista las diferentes bandejas que conforman el sistema y detalla sobre que tipo de información guarda cada bandeja y cual es su equivalente en el sistema bantotal.
			1.3.1.Parametrizaciones:  
		1.4.Control de Calidad de Datos de Bandejas:  No tiene contenido, sirve para agrupar subsecciones hijas
			1.4.1.Programas de Control:  Lista los programas de control asociados a cada bandeja, que se u

In [25]:
import dill as pkl
filename = 'dict_arbol_secciones.pkl'
with open(filename, 'rb') as file:
    trees = pkl.load(file)

In [50]:
from langchain.prompts import PromptTemplate

prompt_rag = PromptTemplate(template=""" 
Sos un asistente en el sistema de migración de Bantotal. 
Dado el siguiente contenido, responder a la pregunta. Responder de forma amplia, ahondando en la mayor cantidad de detalles, pero SIN INVENTAR INFORMACIÓN. Toda la información debe ser recuperada del contenido. 
<contenido>
{contenido}
</contenido>

pregunta: {pregunta}

""", input_variables=["pregunta", "secciones"])

prompt_seccion = PromptTemplate(template=""" 
Sos un asistente en el sistema de migración de Bantotal. 
Debes ayudar al usuario a elegir cual de las secciones es la más apropiada para encontrar la respuesta a una pregunta.

El indice con las secciones es el siguiente:
{secciones}

Y la pregunta es: {pregunta}

Responder UNICAMENTE con un json blob con dos keys "nombre_seccion" y "numero_seccion", donde se encuentra el nombre y el numero (x.x.x) de la seccion más probable para responder la pregunta.

""", input_variables=["pregunta", "secciones"])

In [55]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda


def content_from_section(dicto):
    nro_seccion = dicto["seccion"]["numero_seccion"]
    manual = dicto["manual"]
    num = nro_seccion.split('.')
    seed = trees[manual]
    for x  in num:
        seed = seed.childrens[int(x)-1]
    content = f"Seccion {nro_seccion}: {seed.section} \n\n {seed.content} + \n\n"
    for child,idx in enumerate(seed.childrens):
        content += f"Seccion {nro_seccion}.{idx+1}: {child.section} \n\n {child.content} + \n\n"
    return content

def get_indice(manual):
    return trees[manual].print_index_w_content()

manual = 'Manual Chequeras'
pregunta = "Que campos tiene la bandeja BNJ006?"

chain_get_section = {"pregunta":itemgetter("pregunta"),"secciones": itemgetter("manual") | RunnableLambda(get_indice)} | prompt_seccion | ChatOpenAI(temperature=0.0) | JsonOutputParser()

chain_get_answer = {"pregunta": itemgetter("pregunta"), "contenido": {"seccion":chain_get_section, "manual": itemgetter("manual")} | RunnableLambda(content_from_section)} | prompt_rag | ChatOpenAI(temperature=0.0) | StrOutputParser()

res  = chain_get_answer.invoke({"pregunta":pregunta,"manual":manual})

res

'La bandeja BNJ006 tiene los siguientes campos:\n- Empresa (BcleEmp)\n- Bandeja (BcleBnd)\n- Módulo (BcleMod)\n- Sucursal (BcleSuc)\n- Moneda (BcleMda)\n- Papel (BclePap)\n- Cuenta Cliente (BcleCta)\n- Operación (BcleOpe)\n- Sub Operación (BcleSub)\n- Tipo de Operación (BcleTOp)\n- Fecha (BclE1Fch)\n- Hora (Bcle1Hra)\n- Producto (Bcle1Prd)\n- Boleta (Bcle1Bol)\n- Cheque (Bcle1Chq)\n- Banco (Bcle1Bco)\n- Sucursal del Banco (Bcle1BSuc)\n- Código Postal (Bcle1CP)\n- Cuenta del Librador (Bcle1CtaL)\n- Plaza (Bcle1Pza)\n- Plazo (Bcle1Pzo)\n- Tipo de Cheque (Bcle1TChq)\n- Importe (Bcle1Imp)\n- Estado (Bcle1Est)\n- Moneda del Cheque (Bcle1Mda)\n- Motivo del Vencimiento (Bcle1MtVto)\n- Estado en Proceso de Migracion (Bcle1MEst)'

In [57]:
for chunk in chain_get_answer.stream({"pregunta":pregunta,"manual":manual}):
    print(chunk)


La
 bande
ja
 BN
J
006
 tiene
 los
 siguientes
 campos
:

-
 Empresa
 (
B
cle
Emp
)

-
 Band
e
ja
 (
B
cle
B
nd
)

-
 M
ód
ulo
 (
B
cle
Mod
)

-
 Suc
ursal
 (
B
cle
Suc
)

-
 Mon
eda
 (
B
cle
M
da
)

-
 Pap
el
 (
B
cle
P
ap
)

-
 C
uenta
 Cliente
 (
B
cle
C
ta
)

-
 Oper
ación
 (
B
cle
O
pe
)

-
 Sub
 Oper
ación
 (
B
cle
Sub
)

-
 Tipo
 de
 Oper
ación
 (
B
cle
TO
p
)

-
 Fecha
 (
B
cl
E
1
F
ch
)

-
 H
ora
 (
B
cle
1
H
ra
)

-
 Producto
 (
B
cle
1
Pr
d
)

-
 Bo
leta
 (
B
cle
1
B
ol
)

-
 Che
que
 (
B
cle
1
Ch
q
)

-
 Banco
 (
B
cle
1
B
co
)

-
 Suc
ursal
 del
 Banco
 (
B
cle
1
BS
uc
)

-
 C
ódigo
 Postal
 (
B
cle
1
CP
)

-
 C
uenta
 del
 Lib
rador
 (
B
cle
1
C
ta
L
)

-
 Plaza
 (
B
cle
1
P
za
)

-
 Pl
azo
 (
B
cle
1
P
zo
)

-
 Tipo
 de
 Che
que
 (
B
cle
1
T
Ch
q
)

-
 Import
e
 (
B
cle
1
Imp
)

-
 Estado
 (
B
cle
1
Est
)

-
 Mon
eda
 del
 Che
que
 (
B
cle
1
M
da
)

-
 Mot
ivo
 del
 V
enc
imiento
 (
B
cle
1
Mt
V
to
)

-
 Estado
 en
 Pro
ceso
 de
 M
igr
ación
 (
B
cle
1
ME
st
)

